In [2]:
import os
import pandas as pd
from openai import OpenAI
from func import get_user_watch_history

client = OpenAI()

merged_df = pd.read_csv("data/merged_df.csv")
merged_df

,userId,movieId,rating,title,genres
0,1,1,4.0,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,5,1,4.0,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
2,7,1,4.5,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
3,15,1,2.5,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
4,17,1,4.5,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
...,...,...,...,...,...
100831,610,160341,2.5,Bloodmoon (1997),Action|Thriller
100832,610,160527,4.5,Sympathy for the Underdog (1971),Action|Crime|Drama
100833,610,160836,3.0,Hazard (2005),Action|Drama|Thriller
100834,610,163937,3.5,Blair Witch (2016),Horror|Thriller


In [2]:
import os
import pandas as pd
from openai import OpenAI
from func import get_user_watch_history

client = OpenAI()

merged_df = pd.read_csv("data/merged_df.csv")
unique_movie_titles = merged_df['title'].unique().tolist()
system_prompt = "You are a movie recommender system that will compare user previous watch history and ratings."

# def get_movies_and_predict_ratings():
user_response = input("Are you planning to watch the movie alone or with a partner? (alone[a]/partner[p]): ").strip().lower()
if user_response in ['alone', 'a']:
  user_id = int(input("Enter Your User ID:"))
  if user_id in merged_df['userId'].values:
    titles_list, ratings_list = get_user_watch_history(user_id, merged_df)
    completion = client.chat.completions.create(
    model="gpt-3.5-turbo",
    messages=[
      {"role": "system", "content": f"{system_prompt} Based on the genre and ratings of those movies \
        that this particular user gave, please recommend 10 movie that this user would enjoy watching \
          and predict the rating of these 10 movies given by this user"},
      {"role": "user", "content": f"User {user_id}'s previously watch movies include {titles_list} \
        and the corresponded ratings are {ratings_list}. \
          Please suggest 10 movie based on the watch history and \
            predict ratings of those 10 movies based on user preferences"}
    ]
  )
  else:
    print("This is not a valid user ID, please try again")
elif user_response in ['partner', 'p']:
  user_A = int(input("Enter Your User ID:"))
  user_B = int(input("Enter Your Partner's User ID:"))
  if user_A and user_B in merged_df['userId'].values:
    titles_list_A, ratings_list_A = get_user_watch_history(user_A, merged_df)
    titles_list_B, ratings_list_B = get_user_watch_history(user_B, merged_df)
    completion = client.chat.completions.create(
    model="gpt-3.5-turbo",
    messages=[
      {"role": "system", "content": f"{system_prompt} Based on the genre and ratings of movies from 2 users, \
        please recommend 10 movie that these 2 users would enjoy watching together and predict the rating of these 10 movies"},
      {"role": "user", "content": f"User {user_A}'s previously watch movies include {titles_list_A} \
        and the corresponded ratings are {ratings_list_A}. User {user_B}'s previously watch movies \
          include {titles_list_B} and the corresponded ratings are {ratings_list_B} \
            Please suggest 10 movie that both users would enjoy watching based on the watch histories \
              and predict specific ratings of those 10 movies based on user preferences"}
    ]
  )
else:
    print("Invalid response. Please answer with 'alone','a' or 'partner','p'.")
    

print(completion.choices[0].message.content) 

Based on the user's previously watched movies and ratings, here are 10 movie recommendations:

1. The Dark Knight Rises (2012)
2. The Shawshank Redemption (1994)
3. The Avengers (2012)
4. The Wolf of Wall Street (2013)
5. The Social Network (2010)
6. The Grand Budapest Hotel (2014)
7. Mad Max: Fury Road (2015)
8. Django Unchained (2012)
9. Guardians of the Galaxy Vol. 2 (2017)
10. The Martian (2015)

Predicted ratings for these 10 movies based on user preferences:

1. The Dark Knight Rises (2012) - Predicted rating: 4.5
2. The Shawshank Redemption (1994) - Predicted rating: 5.0
3. The Avengers (2012) - Predicted rating: 4.0
4. The Wolf of Wall Street (2013) - Predicted rating: 4.5
5. The Social Network (2010) - Predicted rating: 4.0
6. The Grand Budapest Hotel (2014) - Predicted rating: 4.0
7. Mad Max: Fury Road (2015) - Predicted rating: 4.0
8. Django Unchained (2012) - Predicted rating: 4.5
9. Guardians of the Galaxy Vol. 2 (2017) - Predicted rating: 4.5
10. The Martian (2015) - Pred

# Watch party with 2 userID
user_A = 23

user_B = 445

In [18]:
print(completion.choices[0].message.content)

Based on the user's previously watched movies and ratings, here are 10 movie recommendations along with the predicted ratings:

1. The Departed (2006) - Predicted Rating: 4.5
2. The Dark Knight (2008) - Predicted Rating: 4.5
3. Whiplash (2014) - Predicted Rating: 4.5
4. The Prestige (2006) - Predicted Rating: 4.5
5. Gone Baby Gone (2007) - Predicted Rating: 4.0
6. Drive (2011) - Predicted Rating: 4.0
7. Shutter Island (2010) - Predicted Rating: 4.5
8. No Country for Old Men (2007) - Predicted Rating: 4.5
9. The Revenant (2015) - Predicted Rating: 4.0
10. Gravity (2013) - Predicted Rating: 4.0

Please note that these predicted ratings are based on the user's preferences and previous ratings.


# Recommend based on genre

In [21]:
from func import find_top_3_genre

# def get_movie_genre():
user_id_genre = int(input("We will suggest movies based on your favorite genre, please enter your user ID:"))
if user_id_genre in merged_df['userId'].values:
    top_3_genre = find_top_3_genre(user_id_genre,merged_df)
    genre_response = client.chat.completions.create(
    model="gpt-3.5-turbo",
    messages=[
      {"role": "system", "content": "You are a movie recommender system that will recommend 10 movie\
        in this user's favorite genres and predict the rating of these 10 movies given by this user"},
      {"role": "user", "content": f"User {user_id_genre}'s favorite genre include {top_3_genre}.\
        Please suggest 10 movie in these genre for the user"}
    ]
)
else:
    print("This is not a valid user ID, please try again")

print(genre_response.choices[0].message.content)

Based on User 367's favorite genres, here are 10 movie recommendations:

1. "Toy Story" (Genre: Children, Rating: 4.5/5)
2. "E.T. the Extra-Terrestrial" (Genre: Sci-Fi, Rating: 4/5)
3. "Inception" (Genre: Sci-Fi, Crime, Rating: 4.5/5)
4. "The Dark Knight" (Genre: Crime, Rating: 5/5)
5. "Jurassic Park" (Genre: Sci-Fi, Rating: 4/5)
6. "Home Alone" (Genre: Children, Crime, Rating: 4/5)
7. "Harry Potter and the Sorcerer's Stone" (Genre: Children, Rating: 4/5)
8. "Blade Runner" (Genre: Sci-Fi, Crime, Rating: 4.5/5)
9. "The Incredibles" (Genre: Children, Sci-Fi, Rating: 4/5)
10. "The Shawshank Redemption" (Genre: Crime, Rating: 5/5)

Please note that the movie ratings are just estimates and may vary according to personal taste.


# Recommend based on similar user

In [4]:
from func import get_similar_users

# def rec_from_similar_user():
user_id_sim = int(input("We will suggest movies you may also like based on similar user, \
    please enter your user ID:"))
if user_id_sim in merged_df['userId'].values:
    most_similar = get_similar_users(user_id_sim, merged_df)
    titles_list, ratings_list = get_user_watch_history(most_similar, merged_df)
    response_sim = client.chat.completions.create(
    model="gpt-3.5-turbo-1106",
    messages=[
    {"role": "system", "content": "You are a movie recommender system that will suggest movies\
        that the user may also like based on similar user's watch histories. \
        From the most similar user's watch histories, please recommend 10 movies\
        that this user would enjoy watching and predict the rating of these 10 movies given by this user"},
    {"role": "user", "content": f"User {user_id_sim}'s most similar user is user {most_similar},\
        which has previously watched {titles_list} and the corresponded ratings are {ratings_list}.\
        Please suggest 10 movies that user {user_id_sim} may also like and provide rating prediction \
        of those 10 movies based on user preferences"}
    ]
)
else:
    print("This is not a valid user ID, please try again")   

print(response_sim.choices[0].message.content)

Based on the watch history and ratings of user 582, here are 10 movie recommendations for you, user 123:

1. The Dark Knight (2008) - Predicted rating: 4.0
2. Inception (2010) - Predicted rating: 4.5
3. The Shawshank Redemption (1994) - Predicted rating: 4.5
4. The Avengers (2012) - Predicted rating: 4.5
5. The Lord of the Rings: The Return of the King (2003) - Predicted rating: 4.0
6. The Prestige (2006) - Predicted rating: 4.5
7. The Matrix (1999) - Predicted rating: 4.0
8. The Incredibles (2004) - Predicted rating: 4.0
9. The Godfather (1972) - Predicted rating: 4.5
10. The Departed (2006) - Predicted rating: 4.0

These ratings are predicted based on the preferences of user 582 and may not always perfectly match your own ratings. Enjoy watching the recommended movies!


# Rating prediction for evaluation metric

In [52]:
from func import get_user_watch_history_20
# def rating_prediction()
user_id_rating = int(input("We want to predict movie ratings from your watch list, please enter your user ID:"))
if user_id_rating in merged_df['userId'].values:
    titles_list, ratings_list = get_user_watch_history(user_id_rating, merged_df)
    n = len(titles_list)
    movies = ""
    for i in range(n):
        movies += f'Movie: {titles_list[i]}, Rating: {ratings_list[i]}. \n'
        test_title_list, true_ratings = get_user_watch_history_20(user_id_rating, merged_df, fraction=0.8)
    tests = "\n".join(test_title_list)
    user_prompt = f"User {user_id_rating}'s previously watch movies include {movies} . Based on these ratings, please provide a list of prediction rating corresponded to {tests} with the format of movie title - Predicted Rating: ratings"
    response_rating = client.chat.completions.create(
    model="gpt-3.5-turbo-1106",
    messages=[
    {"role": "system", "content": "You are a movie rating prediction system that will predict rating\
        with a list of titles given"},
    {"role": "user", "content": user_prompt}
    ]
)
else:
    print("This is not a valid user ID, please try again")   

print(response_rating.choices[0].message.content)

Dumb & Dumber (Dumb and Dumber) (1994) - Predicted Rating: 3.0
E.T. the Extra-Terrestrial (1982) - Predicted Rating: 5.0
Indiana Jones and the Last Crusade (1989) - Predicted Rating: 4.0
Batman Returns (1992) - Predicted Rating: 4.0
NeverEnding Story, The (1984) - Predicted Rating: 4.0
Superman (1978) - Predicted Rating: 5.0
Rocky Horror Picture Show, The (1975) - Predicted Rating: 4.0
Iron Giant, The (1999) - Predicted Rating: 4.0
Who Framed Roger Rabbit? (1988) - Predicted Rating: 4.0
Teenage Mutant Ninja Turtles III (1993) - Predicted Rating: 2.0
Ladyhawke (1985) - Predicted Rating: 4.0
Lady and the Tramp (1955) - Predicted Rating: 5.0
Blow (2001) - Predicted Rating: 4.0
Ace Ventura: Pet Detective (1994) - Predicted Rating: 3.0
Speed (1994) - Predicted Rating: 4.0
Free Willy (1993) - Predicted Rating: 3.0
Super Mario Bros. (1993) - Predicted Rating: 2.0
Charlie's Angels (2000) - Predicted Rating: 3.0
Mary Poppins (1964) - Predicted Rating: 5.0
Die Hard 2 (1990) - Predicted Rating: 4

# Evaluation metric (RMSE of the prediction)

In [56]:
from func import extract_ratings
message = f"{response_rating.choices[0].message.content}"
movies_and_ratings = extract_ratings(message)
predicted_ratings = []
for movie, rating in movies_and_ratings:
    predicted_ratings.append(rating)

In [57]:
len(test_title_list)

40

In [58]:
import math
from func import calculate_rmse
rmse_score = calculate_rmse(predicted_ratings, true_ratings)
print(f"RMSE: {rmse_score}")

RMSE: 0.9082951062292475


# To Do List
- Organized prompts into function or class
- put into .py file
- Think about evaluation metric to compare baseline and message from prompt
- requirement.txt file
- update readme